<a href="https://colab.research.google.com/github/eric15342335/realfill/blob/main/train_realfill.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/eric15342335/realfill
%cd realfill

Cloning into 'realfill'...
remote: Enumerating objects: 316, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 316 (delta 68), reused 35 (delta 28), pack-reused 216 (from 1)
Receiving objects: 100% (316/316), 1.29 MiB | 23.94 MiB/s, done.
Resolving deltas: 100% (139/139), done.
/content/realfill


In [2]:
!curl -L https://github.com/eric15342335/realfill/releases/download/dataset/realfill_data_release_full.zip -o realfill_data_release_full.zip
!unzip -q realfill_data_release_full.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  452M  100  452M    0     0  66.8M      0  0:00:06  0:00:06 --:--:-- 42.9M


In [3]:
!curl -L https://github.com/eric15342335/realfill/releases/download/dataset/jensen_images.zip -o jensen_images.zip
!unzip -q jensen_images.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 43.5M  100 43.5M    0     0  38.0M      0  0:00:01  0:00:01 --:--:-- 82.0M


In [4]:
%pwd
%ls -lh

total 497M
drwxr-xr-x 4 root root 4.0K Apr 22 13:56 data/
-rw-r--r-- 1 root root 3.4K Apr 22 13:56 infer.py
drwxr-xr-x 3 root root 4.0K Apr 22 00:01 jensen_images/
-rw-r--r-- 1 root root  44M Apr 22 13:56 jensen_images.zip
-rw-r--r-- 1 root root 1.1K Apr 22 13:56 LICENSE
drwxr-xr-x 3 root root 4.0K Apr 22 13:56 __MACOSX/
-rw-r--r-- 1 root root  383 Apr 22 13:56 README.md
-rw-r--r-- 1 root root 5.9K Apr 22 13:56 README-Realfill.md
drwxr-xr-x 4 root root 4.0K May 30  2024 realfill_data_release_full/
-rw-r--r-- 1 root root 453M Apr 22 13:56 realfill_data_release_full.zip
-rw-r--r-- 1 root root  119 Apr 22 13:56 requirements.txt
-rw-r--r-- 1 root root  26K Apr 22 13:56 train_realfill.ipynb
-rw-r--r-- 1 root root  37K Apr 22 13:56 train_realfill.py


In [ ]:
!uv pip install -U -r requirements.txt --no-progress

Using Python 3.11.12 environment at: /usr
Resolved 58 packages in 454ms
 Downloaded setuptools
 Downloaded pillow
 Downloaded diffusers
 Downloaded tensorboard
 Downloaded nvidia-cuda-cupti-cu12
 Downloaded nvidia-nvjitlink-cu12
 Downloaded nvidia-cuda-nvrtc-cu12
 Downloaded numpy
 Downloaded xformers
 Downloaded nvidia-curand-cu12
 Downloaded bitsandbytes
 Downloaded nvidia-cusolver-cu12


In [ ]:
from accelerate.utils import write_basic_config
write_basic_config()

In [ ]:
import os

# Toggle switch - set to True for Google Drive storage, False for local storage
USE_DRIVE_STORAGE = True
# --- Precision Toggles ---
# Set only ONE of these to True, or both to False for FP16
USE_FP32 = False # Highest precedence: If True, uses FP32
USE_BF16 = True # Second precedence: If True (and USE_FP32 is False), uses BF16
# If both USE_FP32 and USE_BF16 are False, uses FP16

# --- Determine Active Precision ---
if USE_FP32:
    active_precision = "fp32"
elif USE_BF16:
    active_precision = "bf16"
else:
    active_precision = "fp16" # Default if both toggles are False

# Base directory for Google Drive storage
DRIVE_BASE_DIR = "/content/drive/MyDrive/RealFill"

# Keep your original environment variable setup
os.environ["DATASET"] = "realfill_data_release_full" # realfill_data_release_full, jensen_images
os.environ["MODEL_NAME"] = "stabilityai/stable-diffusion-2-inpainting"
os.environ["BENCHMARK"] = "RealBench"  # RealBench, Qualitative, Custom
os.environ["DATASET_NUMBER"] = "24"

# Construct paths based on the storage toggle
if USE_DRIVE_STORAGE:
    # Use Google Drive for storage
    # Make sure drive is mounted if needed (place this where appropriate in your Colab flow)
    try:
        from google.colab import drive # Check if in Colab
        drive.mount('/content/drive', force_remount=True) # Added force_remount for convenience
    except ImportError:
        print("Not running in Google Colab or drive import failed. Assuming local paths for DRIVE_BASE_DIR.")
        # Potentially fall back to local or handle error depending on expected environment
    base_output_prefix = f'{DRIVE_BASE_DIR}/{os.environ["BENCHMARK"]}-{os.environ["DATASET_NUMBER"]}'
else:
    # Use local storage (e.g., Colab ephemeral storage or local machine)
    base_output_prefix = f'{os.environ["BENCHMARK"]}-{os.environ["DATASET_NUMBER"]}'

# --- Construct suffix for directories based on active precision ---
# Append suffix only for non-default precisions (FP32, BF16)
precision_suffix = ""
if active_precision == "fp32":
    precision_suffix = "-fp32"
elif active_precision == "bf16":
    precision_suffix = "-bf16"

# --- Set output paths using the determined suffix ---
os.environ["OUTPUT_DIR"] = f'{base_output_prefix}-model{precision_suffix}'
os.environ["OUTPUT_IMG_DIR"] = f'{base_output_prefix}-results{precision_suffix}'

# --- Set paths for training data and validation images ---
os.environ["TRAIN_DIR"] = f'{os.environ["DATASET"]}/{os.environ["BENCHMARK"]}/{os.environ["DATASET_NUMBER"]}'
# Ensure TRAIN_DIR exists relative to the potential drive mount or local path
# This assumes DATASET directory is at the root or accessible from DRIVE_BASE_DIR
# Adjust if your DATASET path depends on USE_DRIVE_STORAGE
if USE_DRIVE_STORAGE and not os.path.exists(os.environ["TRAIN_DIR"]):
     # Example: If DATASET path is also on Drive relative to DRIVE_BASE_DIR's parent
     potential_train_dir_on_drive = os.path.join(DRIVE_BASE_DIR, "..", os.environ["DATASET"], os.environ["BENCHMARK"], os.environ["DATASET_NUMBER"])
     if os.path.exists(potential_train_dir_on_drive):
           os.environ["TRAIN_DIR"] = potential_train_dir_on_drive
     # Add more logic here if TRAIN_DIR location is complex

os.environ["VAL_IMG"] = os.path.join(os.environ["TRAIN_DIR"], 'target', 'target.png')
os.environ["VAL_MASK"] = os.path.join(os.environ["TRAIN_DIR"], 'target', 'mask.png')


# Create the necessary output directories
os.makedirs(os.environ["OUTPUT_DIR"], exist_ok=True)
os.makedirs(os.environ["OUTPUT_IMG_DIR"], exist_ok=True)

# --- Set the shell environment variable for the training script ---
# This makes the variable available to the subshell created by !
# Assumes the training script uses `--mixed_precision=bf16` or `--mixed_precision=fp16`
# and no argument (or perhaps --mixed_precision=no) for FP32.
if active_precision == "fp32":
    os.environ['PRECISION_ARG'] = '' # Empty for FP32
elif active_precision == "bf16":
    os.environ['PRECISION_ARG'] = '--mixed_precision=bf16'
else: # fp16
    os.environ['PRECISION_ARG'] = '--mixed_precision=fp16'

# --- Print Statements ---
print(f"Using {'Google Drive' if USE_DRIVE_STORAGE else 'local'} storage")
print(f"Selected Training Precision: {active_precision.upper()}")
print(f"Train directory: {os.environ['TRAIN_DIR']}")
print(f"Validation Image: {os.environ['VAL_IMG']}")
print(f"Validation Mask: {os.environ['VAL_MASK']}")
print(f"Output Model directory: {os.environ['OUTPUT_DIR']}")
print(f"Output Results directory: {os.environ['OUTPUT_IMG_DIR']}")
print(f"Precision argument for script: '{os.environ['PRECISION_ARG']}'")

In [ ]:
!accelerate launch train_realfill.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --train_data_dir=$TRAIN_DIR \
  --output_dir=$OUTPUT_DIR \
  --resolution=512 \
  --train_batch_size=16 \
  --gradient_accumulation_steps=1 \
  --unet_learning_rate=2e-4 \
  --text_encoder_learning_rate=4e-5 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=100 \
  --max_train_steps=2000 \
  --lora_rank=8 \
  --lora_dropout=0.1 \
  --lora_alpha=16 \
  --resume_from_checkpoint="latest" \
  --report_to tensorboard \
  --validation_steps 100 \
  --checkpointing_steps 100 \
  $PRECISION_ARG \
  --use_8bit_adam \
  --set_grads_to_none \
  --enable_xformers_memory_efficient_attention

In [ ]:
!accelerate launch infer.py \
    --model_path=$OUTPUT_DIR \
    --validation_image=$VAL_IMG \
    --validation_mask=$VAL_MASK \
    --output_dir=$OUTPUT_IMG_DIR

In [ ]:
# Zip final inference results
!zip -r9j $OUTPUT_IMG_DIR.zip $OUTPUT_IMG_DIR
# Zip tensorboard logs
!zip -r9D $OUTPUT_DIR-tensorboard.zip $OUTPUT_DIR/logs
%ls